In [1]:
%pwd

u'/Users/yan/Code/avvo_nlp'

In [2]:
from __future__ import print_function
import os, sys, collections
import numpy as np
from numpy import linalg as LA
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nlp_utility import *

Using TensorFlow backend.


In [3]:
fn = '/Users/yan/qa-classifier/qa_recom_refQuestions.csv'
sen = readfile(fn)

In [4]:
len(sen), sen[0]

(128463,
 [2808839,
  112,
  'where would i find and ingressegress easement if it was documented by not on the deed purchasing a home now at closing and title company says there is not recorded easement on the deed at this point everything is complete but this we are 17 days past our closing date because of this problem'])

In [5]:
def tokenizer(texts):
    sentences = []
    for sen_cur in texts:
        words = []
        for i in sen_cur.split(' '):
            if len(i) > 0:
                words.append(i)
        sentences.append(words)
    return sentences

texts = [i[2] for i in sen]
sentences = tokenizer(texts)

In [25]:
print(sentences[:2])

[['where', 'would', 'i', 'find', 'and', 'ingressegress', 'easement', 'if', 'it', 'was', 'documented', 'by', 'not', 'on', 'the', 'deed', 'purchasing', 'a', 'home', 'now', 'at', 'closing', 'and', 'title', 'company', 'says', 'there', 'is', 'not', 'recorded', 'easement', 'on', 'the', 'deed', 'at', 'this', 'point', 'everything', 'is', 'complete', 'but', 'this', 'we', 'are', '17', 'days', 'past', 'our', 'closing', 'date', 'because', 'of', 'this', 'problem'], ['how', 'do', 'i', 'take', 'care', 'of', 'a', 'bench', 'warrant', 'from', 'california', 'and', 'i', 'live', 'in', 'washington', 'state', 'i', 'apparently', 'missed', 'court', 'in', '2011', 'for', 'sentencing', 'for', 'false', 'representation', 'to', 'obtain', 'aid', 'of', '950', 'or', 'more']]


In [7]:
def build_unigram(sentences):
    unigram = []
    for sen_cur in sentences:
        for i in sen_cur:
            if len(i) > 0:
                unigram.append(i)
    return unigram

unigram = build_unigram(sentences)

In [8]:
len(unigram)

12516944

In [9]:
def count_words(count, words, VOCAB_SIZE=20000):
    import collections
    count.extend(collections.Counter(words).most_common(VOCAB_SIZE - 1))
    return count

count = [('UNK', -1)]
count = count_words(count, unigram, VOCAB_SIZE = 40000)

In [10]:
count[:10]

[('UNK', -1),
 ('i', 511602),
 ('the', 450196),
 ('to', 423515),
 ('and', 334535),
 ('a', 333456),
 ('my', 291032),
 ('in', 207049),
 ('for', 187611),
 ('of', 180527)]

In [11]:
def build_dictionary(count):
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return  dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dictionary(count)
sequences = texts_to_sequences(sentences, dictionary, span=4)
print(len(sequences[2]),len(sequences))

47 128463


In [16]:
print(sentences[2])
print(sequences[2])

['how', 'ong', 'can', 'probation', 'office', 'use', 'old', 'revocation', 'against', 'new', 'case', 'i', 'was', 'very', 'unaware', 'or', 'how', 'system', 'work', 'and', 'po', 'duke', 'me', 'to', 'revoking', '12', 'years', 'ago', 'had', 'nothing', 'sense', 'tell', 'aug', 'of', '2016', 'now', 'there', 'using', 'old', 'revocation', 'to', 'set', 'all', 'rules', 'on', 'new', 'case']
[46, 0, 18, 149, 300, 201, 130, 2474, 178, 120, 55, 1, 11, 203, 2792, 28, 46, 850, 80, 4, 1215, 16316, 14, 3, 6406, 469, 57, 126, 42, 291, 1890, 356, 1713, 9, 400, 51, 50, 435, 130, 2474, 3, 457, 69, 1353, 15, 120, 55]


In [26]:
reverse_dictionary[126]

'ago'

In [ ]:
## Matrix manipulation

In [19]:
from numpy import linalg as LA
a = np.array([[0, 3, 4, 2, 5],
              [4, 2, 6, 3, 1],
              [2, 1, 1, 8, 8],
              [6, 6, 3, 2, 6]])

In [21]:
## Normalized Array
a_norm = LA.norm(a, axis=1)  
a_n = a / a_norm.reshape((a.shape[0],1))
a_n

array([[ 0.        ,  0.40824829,  0.54433105,  0.27216553,  0.68041382],
       [ 0.49236596,  0.24618298,  0.73854895,  0.36927447,  0.12309149],
       [ 0.17277369,  0.08638684,  0.08638684,  0.69109474,  0.69109474],
       [ 0.54545455,  0.54545455,  0.27272727,  0.18181818,  0.54545455]])

In [22]:
## Find the index of the top 2 elements in descending order in each row 
sorted_row_idx = np.argsort(a_n, axis=1)[:,-2::][:,::-1]
sorted_row_idx

array([[4, 2],
       [2, 0],
       [4, 3],
       [4, 1]])